In [22]:
import pandas as pd
import requests
import zipfile
from io import StringIO
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import datetime as dt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from scipy.sparse import hstack

ImportError: cannot import name 'joblib' from 'sklearn.externals' (/opt/conda/lib/python3.7/site-packages/sklearn/externals/__init__.py)

# Test Area 1

In [4]:
data = pd.read_csv("https://github.com/Galeforse/DST-Assessment-01/raw/main/Data/KD99_corrected.csv.zip")

In [7]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data.drop('label', axis=1), data['label'], stratify=data['label'], test_size=0.25)

print('Train data')
print(X_train.shape)
print(Y_train.shape)
print('='*20)
print('Test data')
print(X_test.shape)
print(Y_test.shape)

Y_train

Train data
(370515, 41)
(370515,)
Test data
(123506, 41)
(123506,)


13844      normal.
402010      smurf.
450809     normal.
411806      smurf.
42197      normal.
            ...   
324191      smurf.
111312    neptune.
406846      smurf.
109753    neptune.
146786     normal.
Name: label, Length: 370515, dtype: object

In [28]:
protocol = list(X_train['protocol_type'].values)
protocol = list(set(protocol))
print('Protocol types are:', protocol)
print("")

service = list(X_train['service'].values)
service = list(set(service))
print('Service types are:', service)
print("")

flag = list(X_train['flag'].values)
flag = list(set(flag))
print('Flag types are:', flag)

Protocol types are: ['icmp', 'udp', 'tcp']

Service types are: ['link', 'vmnet', 'ftp_data', 'time', 'pop_2', 'Z39_50', 'discard', 'X11', 'login', 'domain_u', 'uucp', 'rje', 'http_443', 'kshell', 'other', 'klogin', 'ecr_i', 'name', 'printer', 'courier', 'eco_i', 'urh_i', 'telnet', 'http', 'remote_job', 'csnet_ns', 'nntp', 'exec', 'imap4', 'ftp', 'mtp', 'netstat', 'hostnames', 'nnsp', 'daytime', 'ssh', 'supdup', 'systat', 'netbios_ns', 'tim_i', 'sunrpc', 'ntp_u', 'iso_tsap', 'efs', 'shell', 'echo', 'urp_i', 'uucp_path', 'finger', 'auth', 'netbios_ssn', 'red_i', 'domain', 'sql_net', 'private', 'netbios_dgm', 'gopher', 'pop_3', 'smtp', 'IRC', 'bgp', 'ldap', 'ctf', 'whois']

Flag types are: ['S0', 'REJ', 'SF', 'SH', 'RSTOS0', 'S2', 'RSTR', 'OTH', 'S3', 'S1', 'RSTO']


# Test Area 2

In [14]:
df_tr = pd.read_csv("https://github.com/Galeforse/DST-Assessment-01/raw/main/Data/kd_train.csv.zip")
df_test = pd.read_csv("https://github.com/Galeforse/DST-Assessment-01/raw/main/Data/kd_test.csv.zip")

In [15]:
attack = df_tr.groupby("label").size().sort_values(ascending=False)
attack = pd.DataFrame(attack)
attack.reset_index(level=0, inplace=True)
attack.columns = ['Attack','Count']
attack

,Attack,Count
0,smurf.,252590
1,neptune.,96598
2,normal.,87549
3,back.,1989
4,satan.,1436
5,ipsweep.,1100
6,portsweep.,937
7,warezclient.,917
8,teardrop.,887
9,pod.,236


In [16]:
attack2 = df_test.groupby("label").size().sort_values(ascending=False)
attack2 = pd.DataFrame(attack2)
attack2.reset_index(level=0, inplace=True)
attack2.columns = ['Attack','Count']
attack2

,Attack,Count
0,smurf.,28200
1,neptune.,10603
2,normal.,9729
3,back.,214
4,satan.,153
5,ipsweep.,147
6,warezclient.,103
7,portsweep.,103
8,teardrop.,92
9,pod.,28


In [17]:
proto = df_tr.groupby("protocol_type").size().sort_values(ascending=False)
proto = pd.DataFrame(proto)
proto.reset_index(level=0, inplace=True)
proto.columns = ['proto','Count']
proto

,proto,Count
0,icmp,255110
1,tcp,171166
2,udp,18343


In [18]:
proto2 = df_test.groupby("protocol_type").size().sort_values(ascending=False)
proto2 = pd.DataFrame(proto2)
proto2.reset_index(level=0, inplace=True)
proto2.columns = ['proto','Count']
proto2

,proto,Count
0,icmp,28492
1,tcp,18899
2,udp,2011


In [20]:
serv = df_tr.groupby("service").size().sort_values(ascending=False)
serv = pd.DataFrame(serv)
serv.reset_index(level=0, inplace=True)
serv.columns = ['service','Count']
serv

,service,Count
0,ecr_i,253139
1,private,99930
2,http,57841
3,smtp,8756
4,other,6492
...,...,...
60,urh_i,13
61,X11,11
62,tim_i,7
63,pm_dump,1


In [21]:
serv2 = df_test.groupby("service").size().sort_values(ascending=False)
serv2 = pd.DataFrame(serv2)
serv2.reset_index(level=0, inplace=True)
serv2.columns = ['service','Count']
serv2

,service,Count
0,ecr_i,28261
1,private,10963
2,http,6452
3,smtp,967
4,other,745
...,...,...
57,sql_net,5
58,sunrpc,5
59,ctf,4
60,tftp_u,1


# Old Code from main file

In [ ]:
df_tr_x = pd.get_dummies(df_tr_x)
df_test_x = pd.get_dummies(df_test_x)

```
service_X11
service_pm_dump
service_red_i
service_tim_i
flag_RSTOS0

service_tftp_u
```

In [ ]:
columns = ["duration","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in"
           ,"lnum_compromised","lroot_shell","lsu_attempted","lnum_root","lnum_file_creations","lnum_shells"
          "lnum_access_files","lnum_outbound_cmds","is_host_login","is_guest_login","count","srv_count"
           ,"serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate"
           ,"srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate"
           ,"dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate"
          ,"dst_host_rerror_rate","dst_host_srv_rerror_rate"]